In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

In [31]:
np.random.seed(42)

In [3]:
Corpus = pd.read_csv("corpus.csv",encoding='latin-1')

In [4]:
print(Corpus['text'])

0        Stuning even for the non-gamer: This sound tr...
1        The best soundtrack ever to anything.: I'm re...
2        Amazing!: This soundtrack is my favorite musi...
3        Excellent Soundtrack: I truly like this sound...
4        Remember, Pull Your Jaw Off The Floor After H...
                              ...                        
9995     A revelation of life in small town America in...
9996     Great biography of a very interesting journal...
9997     Interesting Subject; Poor Presentation: You'd...
9998     Don't buy: The box looked used and it is obvi...
9999     Beautiful Pen and Fast Delivery.: The pen was...
Name: text, Length: 10000, dtype: object


In [5]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [16]:
Encoder = LabelEncoder()
Corpus['label'] = Encoder.fit_transform(Corpus['label'])
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [17]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [18]:
print(Tfidf_vect.vocabulary_)

{'stun': 4268, 'even': 1544, 'sound': 4122, 'track': 4560, 'beautiful': 380, 'paint': 3161, 'mind': 2839, 'well': 4866, 'would': 4952, 'recomend': 3604, 'people': 3233, 'hate': 2064, 'video': 4772, 'game': 1888, 'music': 2930, 'play': 3309, 'cross': 1025, 'ever': 1548, 'best': 417, 'back': 327, 'away': 317, 'crude': 1028, 'take': 4369, 'fresh': 1847, 'step': 4207, 'guitar': 2020, 'orchestra': 3096, 'impress': 2240, 'anyone': 199, 'care': 641, 'listen': 2613, 'soundtrack': 4123, 'anything': 200, 'read': 3570, 'lot': 2657, 'review': 3726, 'say': 3848, 'figure': 1718, 'write': 4957, 'disagree': 1232, 'bit': 438, 'ultimate': 4637, 'masterpiece': 2753, 'timeless': 4504, 'year': 4976, 'beauty': 382, 'simply': 4019, 'refuse': 3630, 'price': 3417, 'tag': 4368, 'pretty': 3412, 'must': 2936, 'go': 1950, 'buy': 593, 'cd': 674, 'much': 2921, 'money': 2878, 'one': 3074, 'feel': 1694, 'worth': 4948, 'every': 1550, 'penny': 3232, 'amaze': 148, 'favorite': 1682, 'time': 4503, 'hand': 2037, 'intense': 

In [19]:
print(Train_X_Tfidf)

  (0, 4976)	0.05685599921431299
  (0, 4957)	0.059160058915746536
  (0, 4718)	0.05209448693481583
  (0, 4717)	0.11014137270597876
  (0, 4611)	0.05935843985521648
  (0, 4518)	0.08601359580961819
  (0, 4503)	0.04494288586681954
  (0, 4465)	0.05054461537707392
  (0, 4399)	0.11350806918387492
  (0, 4324)	0.07344622363343176
  (0, 4022)	0.07152129850993925
  (0, 3848)	0.10904112604819732
  (0, 3726)	0.06311179161415416
  (0, 3720)	0.07432843940711836
  (0, 3657)	0.12408891568693382
  (0, 3638)	0.10278957639905667
  (0, 3579)	0.10462943660574517
  (0, 3576)	0.07124032129875647
  (0, 3519)	0.11769823179100154
  (0, 3510)	0.06815100514035377
  (0, 3444)	0.061322001534693735
  (0, 3427)	0.20659095731580868
  (0, 3426)	0.3624814284509314
  (0, 3267)	0.195683213566516
  (0, 3200)	0.08748452902433917
  :	:
  (6999, 2869)	0.12978311319957622
  (6999, 2734)	0.1396501345965143
  (6999, 2725)	0.07963995854810256
  (6999, 2623)	0.15989271425137205
  (6999, 2577)	0.17409273584271953
  (6999, 2416)	0.1051

In [96]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {'C': [10, 55, 100], 
              'gamma': [1, 0.1, 0.01, 0.001, 'scale'],
              'kernel': ['rbf']}
SVM = svm.SVC()
random_search = RandomizedSearchCV(estimator=SVM, param_distributions=param_grid, cv=5, n_iter=15,n_jobs=-1)

# Fit randomized search to the data
random_search.fit(Train_X_Tfidf, Train_Y)

# Print best parameters found
print("Best parameters found: ", random_search.best_params_)

Best parameters found:  {'kernel': 'rbf', 'gamma': 1, 'C': 10}


In [20]:
svc=svm.SVC(kernel='rbf',gamma=1,C=10)
svc.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = svc.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  84.6
